# Пример использования LightAutoML с датасетом из OpenML

Этот ноутбук демонстрирует базовое использование LightAutoML для решения задачи классификации на датасете из OpenML.

## 📋 Резюме решенных проблем

### Проблемы, которые были исправлены:

1. **Ошибка импорта LightAutoML (TypeError с log_calls)**
   - **Проблема**: `TypeError: cannot set 'get_record_history_wrapper' attribute of immutable type 'object'`
   - **Решение**: Обновлен `log_calls` до версии 0.3.2 и `lightautoml` до версии 0.4.1

2. **Ошибка с CategoricalDtype**
   - **Проблема**: `TypeError: Cannot interpret 'CategoricalDtype(...)' as a data type`
   - **Решение**: Преобразование всех категориальных колонок в строковый тип перед передачей в LightAutoML

3. **Ошибка с psutil/joblib**
   - **Проблема**: `AttributeError: module 'psutil' has no attribute 'Process'`
   - **Решение**: 
     - Установлен `psutil`
     - Отключены `advanced_roles` в параметрах reader (`advanced_roles: False`)
     - Уменьшен `n_jobs` до 1 для стабильности

### Как запускать LightAutoML:

1. **Установка зависимостей**:
   ```bash
   uv pip install lightautoml>=0.4.1 log_calls psutil
   ```

2. **Инициализация**:
   ```python
   from lightautoml.automl.presets.tabular_presets import TabularAutoML
   from lightautoml.tasks import Task
   
   task = Task('binary')  # или 'reg' для регрессии
   automl = TabularAutoML(
       task=task,
       timeout=300,
       cpu_limit=4,
       reader_params={
           'n_jobs': 1,
           'cv': 5,
           'random_state': 42,
           'advanced_roles': False,  # Важно для избежания проблем с psutil
       }
   )
   ```

3. **Подготовка данных**:
   - Преобразуйте категориальные колонки в строковый тип
   - Убедитесь, что target также строковый, если категориальный

4. **Обучение**:
   ```python
   roles = {
       'target': 'target',
       'category': categorical_features
   }
   oof_predictions = automl.fit_predict(train_data, roles=roles, verbose=1)
   ```

5. **Предсказания**:
   ```python
   test_predictions = automl.predict(test_data)
   ```



In [1]:
# Импорт необходимых библиотек
import openml
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import train_test_split

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

print("Все библиотеки успешно импортированы!")

Все библиотеки успешно импортированы!


## 1. Загрузка датасета из OpenML


Выберем популярный датасет для бинарной классификации. Например, dataset_id=31 (credit-g) - классический датасет для предсказания кредитного риска.


In [2]:
# Загрузка датасета из OpenML
dataset_id = 31  # credit-g dataset
print(f"Загрузка датасета с ID={dataset_id} из OpenML...")

dataset = openml.datasets.get_dataset(dataset_id)
print(f"Датасет загружен: {dataset.name}")
print(f"Описание: {dataset.description[:200]}...")

# Получение данных
X, y, categorical_indicator, attribute_names = dataset.get_data(
    target=dataset.default_target_attribute
)

print(f"\nРазмер данных: {X.shape}")
print(f"Количество признаков: {X.shape[1]}")
print(f"Количество категориальных признаков: {sum(categorical_indicator)}")
print(f"Количество классов: {len(np.unique(y))}")
print(f"Распределение классов:\n{pd.Series(y).value_counts()}")


Загрузка датасета с ID=31 из OpenML...
Датасет загружен: credit-g
Описание: **Author**: Dr. Hans Hofmann  
**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)) - 1994    
**Please cite**: [UCI](https://archive.ics.uci.edu/ml/citation_policy....

Размер данных: (1000, 20)
Количество признаков: 20
Количество категориальных признаков: 13
Количество классов: 2
Распределение классов:
class
good    700
bad     300
Name: count, dtype: int64


## 2. Подготовка данных

Преобразуем данные в формат pandas DataFrame и подготовим для LightAutoML.


In [3]:
# Преобразование в pandas DataFrame
if not isinstance(X, pd.DataFrame):
    X = pd.DataFrame(X, columns=attribute_names)

# Добавление целевой переменной
data = X.copy()
data['target'] = y

# Определение категориальных признаков
categorical_features = [attribute_names[i] for i in range(len(attribute_names)) if categorical_indicator[i]]

# Важно: преобразуем все категориальные колонки в строковый тип
# Это необходимо для совместимости с LightAutoML 0.4.1
for col in categorical_features:
    if col in data.columns:
        # Преобразуем CategoricalDtype в обычный строковый тип
        if isinstance(data[col].dtype, pd.CategoricalDtype):
            data[col] = data[col].astype(str)
        elif data[col].dtype == 'object':
            data[col] = data[col].astype(str)

# Также преобразуем target в строковый тип, если он категориальный
if isinstance(data['target'].dtype, pd.CategoricalDtype):
    data['target'] = data['target'].astype(str)

print(f"Категориальные признаки ({len(categorical_features)}): {categorical_features[:5]}...")
print(f"\nПервые строки данных:")
print(data.head())
print(f"\nИнформация о данных:")
print(data.info())


Категориальные признаки (13): ['checking_status', 'credit_history', 'purpose', 'savings_status', 'employment']...

Первые строки данных:
  checking_status  duration                  credit_history  \
0              <0         6  critical/other existing credit   
1        0<=X<200        48                   existing paid   
2     no checking        12  critical/other existing credit   
3              <0        42                   existing paid   
4              <0        24              delayed previously   

               purpose  credit_amount    savings_status employment  \
0             radio/tv         1169.0  no known savings        >=7   
1             radio/tv         5951.0              <100     1<=X<4   
2            education         2096.0              <100     4<=X<7   
3  furniture/equipment         7882.0              <100     4<=X<7   
4              new car         4870.0              <100     1<=X<4   

   installment_commitment     personal_status other_parties  ..

C:\Users\Aleksandr\AppData\Local\Temp\ipykernel_21176\1435568455.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(data[col]):
C:\Users\Aleksandr\AppData\Local\Temp\ipykernel_21176\1435568455.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(data[col]):
C:\Users\Aleksandr\AppData\Local\Temp\ipykernel_21176\1435568455.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(data[col]):
C:\Users\Aleksandr\AppData\Local\Temp\ipykernel_21176\1435568455.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, 

## 3. Разделение на train и test

Разделим данные на обучающую и тестовую выборки.

In [ ]:
# Разделение на train и test
train_data, test_data = train_test_split(
    data, 
    test_size=0.2, 
    random_state=42, 
    stratify=data['target']
)

# Убеждаемся, что категориальные колонки остаются строковыми после разделения
for col in categorical_features:
    if col in train_data.columns:
        if isinstance(train_data[col].dtype, pd.CategoricalDtype):
            train_data[col] = train_data[col].astype(str)
        if isinstance(test_data[col].dtype, pd.CategoricalDtype):
            test_data[col] = test_data[col].astype(str)

print(f"Размер обучающей выборки: {train_data.shape}")
print(f"Размер тестовой выборки: {test_data.shape}")
print(f"\nРаспределение классов в train:")
print(train_data['target'].value_counts())
print(f"\nРаспределение классов в test:")
print(test_data['target'].value_counts())


Размер обучающей выборки: (800, 21)
Размер тестовой выборки: (200, 21)

Распределение классов в train:
target
good    560
bad     240
Name: count, dtype: int64

Распределение классов в test:
target
good    140
bad      60
Name: count, dtype: int64


C:\Users\Aleksandr\AppData\Local\Temp\ipykernel_21176\3916335668.py:12: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(train_data[col]):
C:\Users\Aleksandr\AppData\Local\Temp\ipykernel_21176\3916335668.py:14: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(test_data[col]):


## 4. Инициализация и обучение LightAutoML

Создадим модель LightAutoML и обучим её на обучающей выборке.


In [5]:
# Определение задачи (бинарная классификация)
task = Task('binary')

# Инициализация LightAutoML
# Отключаем advanced_roles, чтобы избежать проблем с psutil/joblib
automl = TabularAutoML(
    task=task,
    timeout=300,  # 5 минут на обучение
    cpu_limit=4,  # Количество ядер CPU
    reader_params={
        'n_jobs': 1,  # Уменьшаем до 1, чтобы избежать проблем с параллельной обработкой
        'cv': 5,  # Количество фолдов для кросс-валидации
        'random_state': 42,
        'advanced_roles': False,  # Отключаем advanced roles для избежания проблем с psutil
    }
)

print("LightAutoML инициализирован!")
print(f"Параметры:")
print(f"  - Задача: {task.name}")
print(f"  - Timeout: 300 секунд")
print(f"  - CPU limit: 4")
print(f"  - CV folds: 5")
print(f"  - Advanced roles: отключены (для избежания проблем с psutil)")


LightAutoML инициализирован!
Параметры:
  - Задача: binary
  - Timeout: 300 секунд
  - CPU limit: 4
  - CV folds: 5
  - Advanced roles: отключены (для избежания проблем с psutil)


## 5. Обучение модели

Обучим модель на обучающей выборке. LightAutoML автоматически определит категориальные признаки и выполнит предобработку данных.


In [6]:
# Определение ролей признаков
roles = {
    'target': 'target',
    'category': categorical_features  # Указываем категориальные признаки
}

print("Начало обучения модели...")
print("Это может занять несколько минут...")

# Обучение модели
oof_predictions = automl.fit_predict(
    train_data,
    roles=roles,
    verbose=1  # Вывод информации о процессе обучения
)

print("\nОбучение завершено!")
print(f"Форма OOF предсказаний: {oof_predictions.data.shape}")


Начало обучения модели...
Это может занять несколько минут...
[00:28:17] Stdout logging level is INFO.
[00:28:17] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[00:28:17] Task: binary

[00:28:17] Start automl preset with listed constraints:
[00:28:17] - time: 300.00 seconds
[00:28:17] - CPU: 4 cores
[00:28:17] - memory: 16 GB

[00:28:17] Train data shape: (800, 21)

[00:28:17] Layer 1 train process start. Time left 299.99 secs
[00:28:17] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[00:28:29] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.7968229166666666
[00:28:29] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[00:28:29] Time left 287.64 secs

[00:28:29] Selector_LightGBM fitting and predicting completed
[00:28:29] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[00:28:30] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = 0.6512127976190476
[00:28:30] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting comp

Optimization Progress: 100%|██████████| 101/101 [00:09<00:00, 11.08it/s, best_trial=24, best_value=0.778]

[00:28:39] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM completed
[00:28:39] Start fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ...


[00:28:39] Fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM finished. score = 0.6627529761904762
[00:28:39] Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM fitting and predicting completed
[00:28:39] Start fitting Lvl_0_Pipe_1_Mod_2_CatBoost ...
[00:28:40] Fitting Lvl_0_Pipe_1_Mod_2_CatBoost finished. score = 0.6469419642857143
[00:28:40] Lvl_0_Pipe_1_Mod_2_CatBoost fitting and predicting completed
[00:28:40] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ... Time budget is 206.17 secs


Optimization Progress: 100%|██████████| 101/101 [00:17<00:00,  5.71it/s, best_trial=89, best_value=0.76]

[00:28:58] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost completed
[00:28:58] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...


[00:28:58] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = 0.6258296130952381
[00:28:58] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[00:28:58] Time left 258.40 secs

[00:28:58] Layer 1 training completed.

[00:28:58] Blending: optimization starts with equal weights. Score = 0.7655357
[00:28:59] Blending: iteration 0: score = 0.8083780, weights = [0.53486145 0.13614719 0.24635072 0.         0.08264064]
[00:28:59] Blending: iteration 1: score = 0.8100074, weights = [0.5583037  0.         0.3126858  0.         0.12901053]
[00:28:59] Blending: iteration 2: score = 0.8100298, weights = [0.557355   0.         0.31613183 0.         0.1265132 ]
[00:28:59] Blending: no improvements for score. Terminated.

[00:28:59] Blending: best score = 0.8100298, best weights = [0.557355   0.         0.31613183 0.         0.1265132 ]
[00:28:59] Automl preset training completed in 41.81 seconds

[00:28:59] Model description:
Final prediction for new objects (level 0) = 
	 0

## 6. Оценка качества на обучающей выборке (OOF)

Оценим качество модели на out-of-fold предсказаниях.


In [7]:
# Получение OOF предсказаний
oof_pred_proba = oof_predictions.data[:, 0]  # Вероятности положительного класса

# Преобразуем строковые метки в числовые для вычисления метрик
# LightAutoML для бинарной классификации использует метку с большим количеством как положительный класс
unique_labels = sorted(train_data['target'].unique())
label_to_num = {label: idx for idx, label in enumerate(unique_labels)}
train_target_num = train_data['target'].map(label_to_num)

# Для roc_auc_score нужно определить, какой класс положительный
# Обычно это класс с индексом 1 (второй класс)
# Если метки 'good' и 'bad', то 'bad' обычно положительный класс
if 'bad' in unique_labels:
    # 'bad' - положительный класс (индекс 1)
    positive_class_idx = unique_labels.index('bad')
else:
    # Берем последний класс как положительный
    positive_class_idx = len(unique_labels) - 1

# Если положительный класс имеет индекс 0, инвертируем вероятности
if positive_class_idx == 0:
    oof_pred_proba = 1 - oof_pred_proba

train_pred = (oof_pred_proba > 0.5).astype(int)

# Вычисление метрик
train_auc = roc_auc_score(train_target_num, oof_pred_proba)
train_accuracy = accuracy_score(train_target_num, train_pred)

print("Метрики на обучающей выборке (OOF):")
print(f"  ROC-AUC: {train_auc:.4f}")
print(f"  Accuracy: {train_accuracy:.4f}")
print(f"\nОтчет о классификации:")
print(classification_report(train_target_num, train_pred, target_names=unique_labels))


Метрики на обучающей выборке (OOF):
  ROC-AUC: 0.8100
  Accuracy: 0.7375

Отчет о классификации:
              precision    recall  f1-score   support

         bad       0.67      0.25      0.36       240
        good       0.75      0.95      0.83       560

    accuracy                           0.74       800
   macro avg       0.71      0.60      0.60       800
weighted avg       0.72      0.74      0.69       800



## 7. Предсказания на тестовой выборке

Сделаем предсказания на тестовой выборке и оценим качество модели.


In [8]:
# Предсказания на тестовой выборке
test_predictions = automl.predict(test_data)

# Получение вероятностей
test_pred_proba = test_predictions.data[:, 0]  # Вероятности положительного класса

# Используем тот же маппинг меток, что и для train
test_target_num = test_data['target'].map(label_to_num)

# Если положительный класс имеет индекс 0, инвертируем вероятности
if positive_class_idx == 0:
    test_pred_proba = 1 - test_pred_proba

test_pred = (test_pred_proba > 0.5).astype(int)

# Вычисление метрик
test_auc = roc_auc_score(test_target_num, test_pred_proba)
test_accuracy = accuracy_score(test_target_num, test_pred)

print("Метрики на тестовой выборке:")
print(f"  ROC-AUC: {test_auc:.4f}")
print(f"  Accuracy: {test_accuracy:.4f}")
print(f"\nОтчет о классификации:")
print(classification_report(test_target_num, test_pred, target_names=unique_labels))


Метрики на тестовой выборке:
  ROC-AUC: 0.7544
  Accuracy: 0.6950

Отчет о классификации:
              precision    recall  f1-score   support

         bad       0.48      0.17      0.25        60
        good       0.72      0.92      0.81       140

    accuracy                           0.69       200
   macro avg       0.60      0.54      0.53       200
weighted avg       0.65      0.69      0.64       200



## 8. Сравнение результатов

Сравним метрики на обучающей и тестовой выборках.


In [9]:
# Создание таблицы сравнения
results = pd.DataFrame({
    'Метрика': ['ROC-AUC', 'Accuracy'],
    'Train (OOF)': [train_auc, train_accuracy],
    'Test': [test_auc, test_accuracy]
})

print("Сравнение результатов:")
print(results.to_string(index=False))
print(f"\nРазница между train и test:")
print(f"  ROC-AUC: {abs(train_auc - test_auc):.4f}")
print(f"  Accuracy: {abs(train_accuracy - test_accuracy):.4f}")


Сравнение результатов:
 Метрика  Train (OOF)     Test
 ROC-AUC      0.81003 0.754405
Accuracy      0.73750 0.695000

Разница между train и test:
  ROC-AUC: 0.0556
  Accuracy: 0.0425


## 9. Дополнительная информация о модели

Посмотрим, какие модели использовал LightAutoML и другую полезную информацию.


In [ ]:
# Информация о модели
print("Информация о модели LightAutoML:")
print(f"  Тип задачи: {automl.task.name}")

# Попытка получить информацию о параметрах модели
try:
    if hasattr(automl, 'timer'):
        print(f"  Время обучения: {automl.timer.time_spent:.2f} секунд")
    if hasattr(automl, 'cpu_limit'):
        print(f"  CPU limit: {automl.cpu_limit}")
    if hasattr(automl, 'model'):
        print(f"\n  Структура модели доступна через automl.model")
        print(f"  Тип модели: {type(automl.model)}")
except Exception as e:
    print(f"  Дополнительная информация: {e}")


Информация о модели LightAutoML:
  Тип задачи: binary


AttributeError: 'TabularAutoML' object has no attribute 'timeout'

## Заключение

Этот пример демонстрирует базовое использование LightAutoML:
- Загрузка датасета из OpenML
- Автоматическая предобработка данных
- Обучение модели с автоматическим подбором алгоритмов
- Оценка качества на обучающей и тестовой выборках

LightAutoML автоматически:
- Определяет типы признаков
- Выполняет feature engineering
- Подбирает оптимальные алгоритмы машинного обучения
- Строит ансамбли моделей
